In [ ]:
from __future__ import print_function, division

from ourwrnet import *
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
from keras.losses import KLDivergence
from sklearn.metrics import accuracy_score
import numpy as np
import keras
from keract import get_activations
import math
import time
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import numpy as np
import sklearn.metrics as metrics

from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

#import wide_residual_network as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Concatenate
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras import optimizers

import torch.nn.functional as F

'''
Function that returns the trainand test data of the CIFAR10 already preprocessed
'''
def getCIFAR10():
    # input image dimensions
    img_rows, img_cols = 32, 32
    num_classes = 10

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    # format of the tensor
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
        input_shape = (3, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
        input_shape = (img_rows, img_cols, 3)

    # convert in to float the images
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # new normalization with z-score
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    print('CIFAR10 loaded')
    return x_train,y_train,x_test,y_test


'''
Function that returns the shape of the CIFAR10 images
'''
def getCIFAR10InputShape():
    img_rows, img_cols = 32, 32
    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_rows, img_cols)
    else:
        input_shape = (img_rows, img_cols, 3)
        
    return input_shape

'''
Function that returns the network to train
'''
def getNetwork(input_shape):
    model, m1, m2, m3=create_wide_residual_network(input_shape, 10, N=2, k=2, dropout=0.)
    return model,m1,m2,m3;

'''
Function that saves a model on the disk
'''
def saveModel(model,filename):
    model_json = model.to_json()
    with open(filename + '.json', 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(filename + '.h5') 

'''
Learning rate scheduler
'''
def lr_schedule(epoch):
    total_epochs = 100
    lrate = 0.1
    if epoch > total_epochs*0.3:
        lrate = 0.02
    if epoch > total_epochs*0.6:
        lrate = 0.004
    if epoch > total_epochs*0.8:
        lrate = 0.0008
    return lrate

'''
Placeholder loss
'''
def useless_loss(y_true,y_pred):
    zer = K.zeros(1)
    
    return zer
    
'''
Function to try to train the network on CIFAR10
'''
def main():
    epochs = 100
    batch_size = 128
    x_train,y_train,x_test,y_test = getCIFAR10()
    
    input_shape = getCIFAR10InputShape()
    print('CIFAR10 shape: ' + str(input_shape))
    
    model,m1,m2,m3 = getNetwork(input_shape)
    
    # define optimizer
    optim_sgd = optimizers.SGD(lr=0.1, decay=5e-4, momentum=0.9)
    
    model.compile(loss="categorical_crossentropy", optimizer=optim_sgd, metrics=["acc"])
    
    m1.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    m2.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    m3.compile(loss=[useless_loss,useless_loss],optimizer='sgd')
    
    datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,  # randomly flip images
    )

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(x_test, y_test),
        workers=4,
        shuffle=True,
        callbacks=[LearningRateScheduler(lr_schedule)])
    
    model_predictions = model.predict(x_train[0:1])
    m1_predictions = m1.predict(x_train[0:1])
    m2_predictions = m2.predict(x_train[0:1])
    m3_predictions = m3.predict(x_train[0:1])
    
    print('Full model predictions:')
    #print(str(model_predictions))
    print("Model 1 predictions:")
    #print(str(m1_predictions))
    print("Model 2 predictions:")
    #print(str(m2_predictions))
    print("Model 3 predictions:")
    #print(str(m3_predictions))
    
    saveModel(model,'wrn_16_2')
    saveModel(m1,'wrn_16_2_layer1')
    saveModel(m2,'wrn_16_2_layer2')
    saveModel(m3,'wrn_16_2_layer3')
    
main()



CIFAR10 loaded
CIFAR10 shape: (32, 32, 3)


/home/test/advanceddeep/our_code/ourwrnet.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(input)
/home/test/advanceddeep/our_code/ourwrnet.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(init)
/home/test/advanceddeep/our_code/ourwrnet.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(x)
/home/test/advanceddeep/our_code/ourwrnet.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras

Wide Residual Network-16-2 created.
Epoch 1/100
391/391 [==============================] - 41s 104ms/step - loss: 2.6344 - acc: 0.3312 - val_loss: 2.4125 - val_acc: 0.4068
Epoch 2/100
391/391 [==============================] - 39s 99ms/step - loss: 1.9057 - acc: 0.4913 - val_loss: 1.6556 - val_acc: 0.5557
Epoch 3/100
391/391 [==============================] - 38s 97ms/step - loss: 1.5682 - acc: 0.5647 - val_loss: 1.4513 - val_acc: 0.6050
Epoch 4/100
391/391 [==============================] - 38s 97ms/step - loss: 1.3714 - acc: 0.6142 - val_loss: 1.5334 - val_acc: 0.6015
Epoch 5/100
391/391 [==============================] - 38s 97ms/step - loss: 1.2475 - acc: 0.6494 - val_loss: 1.2005 - val_acc: 0.6855
Epoch 6/100
391/391 [==============================] - 38s 97ms/step - loss: 1.1641 - acc: 0.6744 - val_loss: 1.2367 - val_acc: 0.6822
Epoch 7/100
391/391 [==============================] - 38s 97ms/step - loss: 1.1052 - acc: 0.6933 - val_loss: 1.2150 - val_acc: 0.6822
Epoch 8/100
391/39

391/391 [==============================] - 40s 101ms/step - loss: 0.4746 - acc: 0.8857 - val_loss: 0.6071 - val_acc: 0.8523
Epoch 62/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4609 - acc: 0.8888 - val_loss: 0.5683 - val_acc: 0.8617
Epoch 63/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4523 - acc: 0.8940 - val_loss: 0.6066 - val_acc: 0.8544
Epoch 64/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4502 - acc: 0.8937 - val_loss: 0.6602 - val_acc: 0.8398
Epoch 65/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4498 - acc: 0.8945 - val_loss: 0.5929 - val_acc: 0.8579
Epoch 66/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4444 - acc: 0.8956 - val_loss: 0.5758 - val_acc: 0.8623
Epoch 67/100
391/391 [==============================] - 38s 97ms/step - loss: 0.4489 - acc: 0.8937 - val_loss: 0.5419 - val_acc: 0.8697
Epoch 68/100
391/391 [==============================] - 38s 